In [1]:
import sys
sys.path.insert(1, "./rnn")
import train
import model


In [126]:
# need stuff from main.py to test transformation into Tensors

def normalize(column_name, df):
    std = df[column_name].std()
    norm_col = df[column_name].apply(lambda x: x - std)
    df[column_name] = norm_col

# builds the labels and vectorizations of given data
#if you want to fool around with including/excluding certain features and whatnot, this is the place to do it

def labels_and_vectors(file, index=0):
    df = pd.read_pickle(file)
    
    wordlist = VulgarExtractor.vulgarWords("feature-extraction/vulgar-extractor/badwords.txt") 
    dftext = df[['text']]
    result = dftext.applymap(lambda x: VulgarExtractor.containsVulgar(x,wordlist))
    df['isVulgar'] = result

    #word_embeddings = [ee.tweetVec(tagged_line) for tagged_line in df['text']]
    textlist = [txt.replace('\n','') for txt in df['text'].tolist()]
    tagged_sents = TwitterParser.tag(textlist)
    df['POS'] = tagged_sents

    processed_sents = []
    for tagged_sent in df['POS']:
        processed_words = []
        for word, tag in tagged_sent:
            if tag == 'U':
                processed_words.append('someurl')
            elif tag == '@':
                processed_words.append('@someuser')
            else:
                processed_words.append(word)
        sent = ' '.join(processed_words)
        processed_sents.append(sent)
    df['text'] = processed_sents

    word_counts = [TwitterParser.word_count(tagged_line) for tagged_line in df['POS']]
    pos_count_list = [TwitterParser.pos_counts(tagged_line) for tagged_line in df['POS']]
    contains_adjs = [TwitterParser.contains_adjectives(tagged_line) for tagged_line in df['POS']]
    contains_urls = [TwitterParser.contains_url(tagged_line) for tagged_line in df['POS']]
    contains_emojis = [TwitterParser.contains_emoji(tagged_line) for tagged_line in df['POS']]
    contains_abbrevs = [TwitterParser.contains_abbreviation(tagged_line) for tagged_line in df['POS']]

    df['wordCount'] = word_counts
    df['posCounts'] = pos_count_list
    df['containsAdjective'] = contains_adjs
    df['containsURL'] = contains_urls
    df['containsEmoji'] = contains_emojis
    df['containsAbbreviation'] = contains_abbrevs
    #df['wordEmbedding'] = word_embeddings


    for i, tag in enumerate(TwitterParser.tagset):
        tag_counts = []
        for pos_counts in df['posCounts']:
            tag_counts.append(pos_counts[i])
        column_name = 'num_' + tag
        df[column_name] = tag_counts
        normalize(column_name, df)
        
    # Changes "true"/"false"/"unverified" to numeric values, just like the in the early cells

    df.loc[df.classification == 'true', 'classification'] = 1
    df.loc[df.classification == 'false', 'classification'] = 2
    df.loc[df.classification == 'unverified', 'classification'] = 0
    # getting the labels

    #removed containsURL
    attributes = ['isVulgar', 'containsAdjective', 'containsEmoji', 'containsAbbreviation', 'wordCount']
    for tag in TwitterParser.tagset:
        attributes.append('num_' + tag)

    labels = df['classification']
    labels = [l for l in labels]
    labels = np.array(labels)


    # getting the values as a list of lists
    values = df[attributes].values.tolist()
    word_embedding_values = df['wordEmbedding'].values.tolist()


    #Below puts the tweet ID as a feature. Comment this out if you aren't using tweetID
    ###for i,index in enumerate(df.index):
    ###    dev_values[i].append(int(index))


# UNCOMMENT THIS IN ORDER TO INCOPORATE WORD_EMBEDDINGS AGAIN
    #for i,d in enumerate(word_embedding_values):
     #   values[i].extend(d)

    values = np.array(values)
    if index == 1:
        return df.index, values

    return labels, values


In [265]:
from FileReader import FileReader
classInstance = FileReader
classInstance.get_dataframe() #IMPORTANT:  saves a pickle to output/simple or output/full. 
import sys
sys.path.insert(1, "./feature-extraction/embed-extractor")
#from EmbedExtractor import EmbedExtractor
sys.path.insert(1, "./feature-extraction/vulgar-extractor")
from VulgarExtractor import VulgarExtractor
sys.path.insert(1, "./feature-extraction/twitter-parser")
from TwitterParser import TwitterParser
import classifiers
import pandas as pd
import numpy as np
import json
#ee = EmbedExtractor()





 553480082996879360

 581153923987206146

 500298588992593920

 552788945017516032

 524948206023880704

 553561170637238272

 758159624122097664

 767725956706414592

 580352273001410560

 581359544682614784

 763098277986209792

 581290271997968384

 775057555865206784

 500280249629036544

 498293668655423488

 774991078265094144

 553553288625672192

 524961721744900097

 524941720249978880

 544294893146091520

 768859780240773121

 764927075522260992

 544274544174071809

 498486826269548545

 544315472075042818

 524923293711998976

 769988636754505729

 524936793633083394

 553480082996879360

 581153923987206146

 500298588992593920

 552788945017516032

 524948206023880704

 553561170637238272

 758159624122097664

 767725956706414592

 580352273001410560

 581359544682614784

 763098277986209792

 581290271997968384

 775057555865206784

 500280249629036544

 498293668655423488

 774991078265094144

 553553288625672192

 524961721744900097

 524941720249978880

 54429489314

AttributeError: 'list' object has no attribute 'head'

In [127]:
tr_labels, tr_values = labels_and_vectors('output/simple/train_data_simple.pickle')
indices, dev_values,= labels_and_vectors('output/simple/dev_data_simple.pickle', index=1)

In [128]:
## convert to tensor Variable here
import torch
import torch.nn as nn
import helpers
from torch.autograd import Variable



In [246]:
n_epochs = 2000
hidden_size = 20
learning_rate = 0.001


decoder = model.RNN(len(tr_values[0]), hidden_size=hidden_size, output_size =3)
# Loss function can be changed as an argument?
decoder_optimizer = torch.optim.SGD(decoder.parameters(), learning_rate)

criterion = nn.NLLLoss()


SyntaxError: invalid syntax (<ipython-input-246-f2179da2ca41>, line 2)

In [247]:
tvs = [arr.tolist() for arr in tr_values]
dvs = [arr.tolist() for arr in tr_values]

tls = [l.item() for l in tr_labels]


In [248]:
tvs[0]

[0.0,
 0.0,
 0.0,
 0.0,
 15.0,
 1.2109006780606513,
 0.39595863979681123,
 -0.20718062552047917,
 0.344229689792404,
 -0.2572742445336669,
 -0.21372359007418834,
 0.0,
 1.2859161772475212,
 -0.9559925467866434,
 -0.7252163737807164,
 -0.1586364050526587,
 1.8704734362120636,
 1.619042495490964,
 -0.42161521306517785,
 -0.2894338912872113,
 -0.15863640505265839,
 0.0,
 0.2610795953247669,
 -0.37429841758257326,
 -0.22428527680566782,
 -0.7620584777480716,
 0.0,
 -0.6633118686714934,
 0.5677896821287431,
 -0.13457721463346745]

In [249]:
def vector_to_tensor(vector):
    tensor = torch.zeros(1, len(tvs[0]))
    for j in range(len(vector)):
        tensor[0][j] = vector[j]
    return tensor


In [262]:
tensor = vector_to_tensor(tvs[141])

TypeError: list indices must be integers or slices, not str

In [251]:
def train_all():
    for i, label in enumerate(tls):
        tensor = vector_to_tensor(tvs[i])  
        category_tensor = Variable(torch.LongTensor([label]))
        line_tensor = Variable(torch.FloatTensor(tensor))
        output, loss = train.train(category_tensor, line_tensor, decoder)
        decoder_optimizer.step()


In [252]:
#from the pytorch tutroial


# Print epoch number, loss, name and guess
# validation_loss = 0
# for key in category_lines_val.keys():
# lines = category_lines_val[key]
#         #total_count += len(lines)
#             for ln in lines:
#             output = evaluate(Variable(line_to_tensor(ln)))
#             category_tensor = Variable(torch.LongTensor([all_categories.index(key)]))
#             validation_loss += criterion(output, category_tensor)
#     validation_losses.append(validation_loss/900)

#     guess, guess_i = category_from_output(output)
#     correct = '✓' if guess == category else '✗ (%s)' % category
#     print('%d %d%% (%s) %.4f %s / %s %s' % (epoch, epoch / n_epochs * 100, time_since(start), loss, line, guess, correct))
for i in range(100):
    train_all()


predictions=[]

for d in dvs:
    tensor = vector_to_tensor(d)  
    line_tensor = Variable(torch.FloatTensor(tensor))
    output = decoder.predict(line_tensor)
    predictions.append(output)
    

#print(predictions)




In [253]:
predictions

[
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  2
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  1
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  1
 [torch.LongTensor of size 1x1], 
  1
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  1
 [torch.LongTensor of size 1x1], 
  1
 [torch.LongTensor of size 1x1], 
  1
 [torch.LongTensor of size 1x1], 
  1
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  0
 [torch.LongTensor of size 1x1], 
  1
 [torc

In [254]:
ps = []

for i, p in enumerate(predictions):
    p = p[0][0]
    if p == 2:
        ps.append('false')
    if p == 1:
        ps.append('true')
    if p == 0:
        ps.append('unverified')

pred_dict = {index:(ps[i],1) for i,index in enumerate(indices)}
pred_dict

{'580319078155468800': ('unverified', 1),
 '580319184652890113': ('unverified', 1),
 '580320684305416192': ('unverified', 1),
 '580321156508577792': ('unverified', 1),
 '580322453928431617': ('unverified', 1),
 '580323060533764097': ('unverified', 1),
 '580324027715063808': ('false', 1),
 '580325090367315968': ('unverified', 1),
 '580326222107951104': ('unverified', 1),
 '580331561398108160': ('true', 1),
 '580332109782466561': ('unverified', 1),
 '580333763512705025': ('unverified', 1),
 '580333909008871424': ('true', 1),
 '580339547269144576': ('true', 1),
 '580339825649291264': ('unverified', 1),
 '580340476949086208': ('unverified', 1),
 '580348081100734464': ('true', 1),
 '580360165540642816': ('true', 1),
 '580371845997682688': ('true', 1),
 '580882341880446977': ('true', 1),
 '581047170637381632': ('unverified', 1),
 '581063377226637312': ('unverified', 1),
 '581293286268129280': ('unverified', 1),
 '581386094337474560': ('unverified', 1),
 '581473088249958400': ('unverified', 1

In [255]:
with open('output/rnn/tr_test.json', 'w') as outfile:
    json.dump(pred_dict, outfile)

In [256]:
!python3 scorer/score.py data/semeval2017-task8-dataset/traindev/rumoureval-subtaskB-dev.json output/rnn/tr_test.json

25 entries in reference file
matching entry: 580325090367315968
matching entry: 580348081100734464
matching entry: 580324027715063808
matching entry: 580319184652890113
matching entry: 580333909008871424
matching entry: 580321156508577792
matching entry: 580320684305416192
matching entry: 580333763512705025
matching entry: 580340476949086208
matching entry: 580339825649291264
matching entry: 580360165540642816
matching entry: 580322453928431617
matching entry: 580882341880446977
matching entry: 580326222107951104
matching entry: 581473088249958400
matching entry: 580371845997682688
matching entry: 580331561398108160
matching entry: 581047170637381632
matching entry: 581293286268129280
matching entry: 580332109782466561
matching entry: 580319078155468800
matching entry: 580339547269144576
matching entry: 581386094337474560
matching entry: 581063377226637312
matching entry: 580323060533764097
25 matched entries in submission
25 entries in reference file
veracity accuracy: 0.28
Micro F1: 